In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [23]:
year = 1985
url = "https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=KPOP&chartDate={}&moved=Y".format(year)
headers = {
    "User-Agent" : UserAgent().chrome
}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [3]:
dom = BeautifulSoup(response.text, 'html.parser')
elements = dom.select("#frm > table > tbody tr")
element = elements[0]

In [4]:
datas = []
for element in elements:
    data = {
        "title" : element.select_one('a > .odd_span').text.split('상')[0].strip(),
        "artist" : element.select_one('div > .fc_mgray').text,
        "link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
    }
    datas.append(data)

In [5]:
song_df = pd.DataFrame(datas)
song_df.tail()

,artist,link,title
94,김연자,https://www.melon.com/song/detail.htm?songId=1...,영동부르스
95,김학래,https://www.melon.com/song/detail.htm?songId=2...,사랑과 약속
96,황선형,https://www.melon.com/song/detail.htm?songId=1...,물망초
97,민지,https://www.melon.com/song/detail.htm?songId=9...,당신이 주는 것
98,이리,https://www.melon.com/song/detail.htm?songId=9...,어제


In [ ]:
### crawling lyrics of each song. Same process as how we gathered the genre of each song

In [6]:
link = song_df.loc[95, 'link']
print(link)

https://www.melon.com/song/detail.htm?songId=2507200


In [28]:
headers = {
    "User-Agent" : UserAgent().chrome
}
response = requests.get(link, headers=headers)

dom = BeautifulSoup(response.content, 'html.parser')
response

<Response [200]>

In [122]:
elements = dom.select_one('.wrap_lyric').text.replace('\r', '').replace('\n', '').replace('\t', '').split('펼치기')[0]
if elements == '[가사 준비중] 멜론 회원 여러분! 가사 등록을 기다리고 있어요.가사등록하기':
    elements = '-'
    
elements

'-'

In [ ]:
# 가사 수집 함수
### also had to gather by dividing years because the site would block us if too much data was crawled at ones

In [8]:
def get_lyrics(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    elements = dom.select_one('.wrap_lyric').text.replace('\r', '').replace('\n', '').replace('\t', '').split('펼치기')[0]
    if elements == '[가사 준비중] 멜론 회원 여러분! 가사 등록을 기다리고 있어요.가사등록하기':
        elements = '-'
        
    return elements

In [62]:
first_df = pd.read_csv('./datas/first.csv', index_col=[0])
first_df.tail()

,Artist,Link,Title,Year,Genre
390,김정호,https://www.melon.com/song/detail.htm?songId=4...,푸른 하늘 아래로,1975,포크/블루스
391,김정호,https://www.melon.com/song/detail.htm?songId=1...,하얀 나비 (Narr. 이윤선),1975,성인가요
392,송대관,https://www.melon.com/song/detail.htm?songId=5...,해뜰 날,1975,성인가요
393,김세환,https://www.melon.com/song/detail.htm?songId=3...,화가 났을까,1975,포크/블루스
394,정미조,https://www.melon.com/song/detail.htm?songId=1...,휘파람을 부세요,1975,발라드


In [63]:
first_df['Lyrics'] = first_df['Link'].apply(get_lyrics)
first_df.tail()

,Artist,Link,Title,Year,Genre,Lyrics
390,김정호,https://www.melon.com/song/detail.htm?songId=4...,푸른 하늘 아래로,1975,포크/블루스,1.내가 가면 너도 갈래 저 멀리 푸른 하늘 아래로 내가 울면 너도 울고 따라갈래 ...
391,김정호,https://www.melon.com/song/detail.htm?songId=1...,하얀 나비 (Narr. 이윤선),1975,성인가요,음- 생각을 말아요 지나간 일들을 음- 그리워 말아요 떠나갈 임인데 꽃잎은 시들어요...
392,송대관,https://www.melon.com/song/detail.htm?songId=5...,해뜰 날,1975,성인가요,꿈을 안고 왔단다내가 왔단다슬픔도 괴로움도 모두 모두 비켜라안 되는 일 없단다 노력...
393,김세환,https://www.melon.com/song/detail.htm?songId=3...,화가 났을까,1975,포크/블루스,손을 뿌리치면서 집에 가야겠다고화를 발칵 뛰어가는 모습이건 이런것이고 저건 저런것이...
394,정미조,https://www.melon.com/song/detail.htm?songId=1...,휘파람을 부세요,1975,발라드,제가 보고 싶을 땐두 눈을 꼭 감고나즈막히 소리 내어휘파람을 부세요외롭다고 느끼실 ...


In [34]:
second_df = pd.read_csv('./datas/second.csv', index_col=[0])
second_df.tail()

,Artist,Link,Title,Year,Genre
478,김연자,https://www.melon.com/song/detail.htm?songId=1...,영동부르스,1985,"발라드, 성인가요"
479,김학래,https://www.melon.com/song/detail.htm?songId=2...,사랑과 약속,1985,발라드
480,황선형,https://www.melon.com/song/detail.htm?songId=1...,물망초,1985,성인가요
481,민지,https://www.melon.com/song/detail.htm?songId=9...,당신이 주는 것,1985,성인가요
482,이리,https://www.melon.com/song/detail.htm?songId=9...,어제,1985,성인가요


In [35]:
second_df['Lyrics'] = second_df['Link'].apply(get_lyrics)
second_df.tail()

,Artist,Link,Title,Year,Genre,Lyrics
478,김연자,https://www.melon.com/song/detail.htm?songId=1...,영동부르스,1985,"발라드, 성인가요",헤어지기 싫어서애태우던 그날 밤피 눈물에 얼룩진그대의 모습어차피 떠나야할인연이기에미...
479,김학래,https://www.melon.com/song/detail.htm?songId=2...,사랑과 약속,1985,발라드,-
480,황선형,https://www.melon.com/song/detail.htm?songId=1...,물망초,1985,성인가요,한없이 흐르는 구름이 되어 애？은 사연을 비되어 전할까 님이여 님이여 나를 잊지 마...
481,민지,https://www.melon.com/song/detail.htm?songId=9...,당신이 주는 것,1985,성인가요,-
482,이리,https://www.melon.com/song/detail.htm?songId=9...,어제,1985,성인가요,지나간 날 우리는얼마나 사랑했는가너는 나를 위했고나는 너를 위했었지가야 할 길이라면...


In [70]:
third_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
third_df = third_df[third_df['Year'].isin([1986, 1987, 1988, 1989])] 
third_df.tail()

,Artist,Genre,Link,Title,Year
1273,이브,록/메탈,https://www.melon.com/song/detail.htm?songId=1...,히말라야,1989
1274,한돌,포크/블루스,https://www.melon.com/song/detail.htm?songId=1...,쓸쓸한 사람,1989
1275,임지훈,포크/블루스,https://www.melon.com/song/detail.htm?songId=1...,친구에게,1989
1276,김상배,성인가요,https://www.melon.com/song/detail.htm?songId=1...,안돼요 안돼,1989
1277,이윤수,발라드,https://www.melon.com/song/detail.htm?songId=2...,그대 곁으로,1989


In [79]:
third_df['Lyrics'] = third_df['Link'].apply(get_lyrics)
third_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
1273,이브,록/메탈,https://www.melon.com/song/detail.htm?songId=1...,히말라야,1989,-
1274,한돌,포크/블루스,https://www.melon.com/song/detail.htm?songId=1...,쓸쓸한 사람,1989,머물고 싶은 삶을 찾아서 나는 방랑의 길을 떠나가지만 떠도는 것은 아니라네나에게도 ...
1275,임지훈,포크/블루스,https://www.melon.com/song/detail.htm?songId=1...,친구에게,1989,소리없는 밤비가 내리는 밤이면휘저으며 스쳐가는 친구의 영상이꿈을 먹고 살자하던 친구...
1276,김상배,성인가요,https://www.melon.com/song/detail.htm?songId=1...,안돼요 안돼,1989,무슨말을 해야만이내게로 오시렵니까아무말을 하지 않는이내맘을 모르시나요이렇다 할 말도...
1277,이윤수,발라드,https://www.melon.com/song/detail.htm?songId=2...,그대 곁으로,1989,-


In [73]:
fourth_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
fourth_df = fourth_df[fourth_df['Year'].isin([1990, 1991, 1992, 1993])]
fourth_df.tail()

,Artist,Genre,Link,Title,Year
1670,양수경,발라드,https://www.melon.com/song/detail.htm?songId=5...,잊어야 할 때,1993
1671,김원준,댄스,https://www.melon.com/song/detail.htm?songId=5...,아무것도 몰라,1993
1672,김종서,록/메탈,https://www.melon.com/song/detail.htm?songId=8...,새가 되어가리,1993
1673,푸른하늘,발라드,https://www.melon.com/song/detail.htm?songId=1...,내게 영원히,1993
1674,양수경,발라드,https://www.melon.com/song/detail.htm?songId=5...,한번 더 생각해봐요,1993


In [81]:
fourth_df['Lyrics'] = fourth_df['Link'].apply(get_lyrics)
fourth_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
1670,양수경,발라드,https://www.melon.com/song/detail.htm?songId=5...,잊어야 할 때,1993,문득 너를 바라 볼때면예전 같지 않은 낯설움을느낄 수 있지나 이제는 너와 함께한시간...
1671,김원준,댄스,https://www.melon.com/song/detail.htm?songId=5...,아무것도 몰라,1993,가죽점퍼에 블루진인생을 아는 것처럼하지만 나는아무것도 모르지이제 우린사랑이 필요한거...
1672,김종서,록/메탈,https://www.melon.com/song/detail.htm?songId=8...,새가 되어가리,1993,저멀리 날아가는 새야들판을 날아 어디로 가는지끝없이 날아가는 새야너혼자 외로이 떠나...
1673,푸른하늘,발라드,https://www.melon.com/song/detail.htm?songId=1...,내게 영원히,1993,조금도 슬픈 척 하지 않았지기다렸다는듯 웃고 말았어오히려 내 앞에선너의 모습이 조금...
1674,양수경,발라드,https://www.melon.com/song/detail.htm?songId=5...,한번 더 생각해봐요,1993,그 무슨말을 하나요 지금 나에게그대를 향한 나의 마음 너무 슬퍼이대로 나를 떠나게 ...


In [92]:
fifth_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
fifth_df = fifth_df[fifth_df['Year'].isin([1994, 1995, 1996, 1997])]
fifth_df.tail()

,Artist,Genre,Link,Title,Year
2069,여행스케치,"발라드, 포크/블루스",https://www.melon.com/song/detail.htm?songId=7...,진실에 관하여,1997
2070,영턱스클럽,댄스,https://www.melon.com/song/detail.htm?songId=7...,질투,1997
2071,김지현,댄스,https://www.melon.com/song/detail.htm?songId=6...,Cat`s Eye,1997
2072,DJ DOC,댄스,https://www.melon.com/song/detail.htm?songId=9397,거짓말이야,1997
2073,주주 클럽,록/메탈,https://www.melon.com/song/detail.htm?songId=7...,나는 나,1997


In [93]:
fifth_df['Lyrics'] = fifth_df['Link'].apply(get_lyrics)
fifth_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
2069,여행스케치,"발라드, 포크/블루스",https://www.melon.com/song/detail.htm?songId=7...,진실에 관하여,1997,많은 시간이 흘러도 변하지 않는다른 믿음을간직해줘 넌기억해 이 노래처럼아주 조심스럽...
2070,영턱스클럽,댄스,https://www.melon.com/song/detail.htm?songId=7...,질투,1997,웃지 못할 사건 또 잊지 못할 사건나 평생 후회할뻔 했던 사랑 다시 찾은 얘기여자들...
2071,김지현,댄스,https://www.melon.com/song/detail.htm?songId=6...,Cat`s Eye,1997,유난히 나를 잡던 그날밤 너를 떠나지 않는건데 난 몰랐던 거야 흔들리고 있는 널 이...
2072,DJ DOC,댄스,https://www.melon.com/song/detail.htm?songId=9397,거짓말이야,1997,난 키도 작고 잘생긴 얼굴도 아냐돈이 많은 것도 아냐좋은 차가 있는 것도 아냐하지만...
2073,주주 클럽,록/메탈,https://www.melon.com/song/detail.htm?songId=7...,나는 나,1997,떼떼떼떼떼-왜 내가 아는 저 많은 사람은 사랑의 과걸 잊는걸까좋았었던 일도 많았을텐...


In [105]:
sixth_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
sixth_df = sixth_df[sixth_df['Year'].isin([1998, 1999, 2000, 2001])]
sixth_df.tail()

,Artist,Genre,Link,Title,Year
2468,화요비,R&B/Soul,https://www.melon.com/song/detail.htm?songId=3743,그런일은,2001
2469,서문탁,록/메탈,https://www.melon.com/song/detail.htm?songId=5...,사미인곡,2001
2470,god,댄스,https://www.melon.com/song/detail.htm?songId=1...,니가 필요해,2001
2471,김조한,R&B/Soul,https://www.melon.com/song/detail.htm?songId=1...,오늘까지만,2001
2472,조성모,댄스,https://www.melon.com/song/detail.htm?songId=1...,다짐,2001


In [106]:
sixth_df['Lyrics'] = sixth_df['Link'].apply(get_lyrics)
sixth_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
2468,화요비,R&B/Soul,https://www.melon.com/song/detail.htm?songId=3743,그런일은,2001,너무나 멀어보여요이렇게 가까이 있는데도언제나 나를 안아주던 따스한인사도 잊은건가요내...
2469,서문탁,록/메탈,https://www.melon.com/song/detail.htm?songId=5...,사미인곡,2001,만백성에게 고하노니사랑하며 살지어다대체 무슨 일을 하관데사랑하지 않고 살아가오쉬었다...
2470,god,댄스,https://www.melon.com/song/detail.htm?songId=1...,니가 필요해,2001,니가 필요해가지마 제발 제발왜 왜 날 떠나려고 해난 난 아직도 니가 필요해다 다시 ...
2471,김조한,R&B/Soul,https://www.melon.com/song/detail.htm?songId=1...,오늘까지만,2001,이젠 모두 잊었니 정말 나를 잊어버렸니왜 슬픈 예감은 틀린적 없는지꿈에 날 찾아와 ...
2472,조성모,댄스,https://www.melon.com/song/detail.htm?songId=1...,다짐,2001,그때 내가 아니야니 얼굴만 봐도 눈물짓던그런 표정 하지마잔인한 너에게 후회 뿐야널 ...


In [123]:
seventh_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
seventh_df = seventh_df[seventh_df['Year'].isin([2002, 2003, 2004, 2005])]
seventh_df.tail()

,Artist,Genre,Link,Title,Year
2867,거북이,댄스,https://www.melon.com/song/detail.htm?songId=6...,빙고 (아싸),2005
2868,전혜빈,댄스,https://www.melon.com/song/detail.htm?songId=9...,2 A.M (2:00am),2005
2869,슈가,댄스,https://www.melon.com/song/detail.htm?songId=9...,현명한 이별,2005
2870,나윤권,발라드,https://www.melon.com/song/detail.htm?songId=9...,기대,2005
2871,클래지콰이,"일렉트로니카, 국내드라마",https://www.melon.com/song/detail.htm?songId=8...,She Is,2005


In [124]:
seventh_df['Lyrics'] = seventh_df['Link'].apply(get_lyrics)
seventh_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
2867,거북이,댄스,https://www.melon.com/song/detail.htm?songId=6...,빙고 (아싸),2005,아 싸Ladies and gentlemen 아싸 또 왔다 나아싸 또 왔다 나기분 좋...
2868,전혜빈,댄스,https://www.melon.com/song/detail.htm?songId=9...,2 A.M (2:00am),2005,HI 너의 입술 Woo 미치겠어 baby steppin steppin 찐하게 kis...
2869,슈가,댄스,https://www.melon.com/song/detail.htm?songId=9...,현명한 이별,2005,다시 돌아와 흔들지 말고 없는 사랑을 이어가지 말고 더 나쁜 모습 보기 전에 이쯤에...
2870,나윤권,발라드,https://www.melon.com/song/detail.htm?songId=9...,기대,2005,나 먼저 돌아서야 하는데괜찮은 척 웃고 있는데내 가슴은 싫다고 붙잡으라고눈물을 만들...
2871,클래지콰이,"일렉트로니카, 국내드라마",https://www.melon.com/song/detail.htm?songId=8...,She Is,2005,숨겨왔던 나의 수줍은 마음 모두 네게 줄께(예이예예~)차가운 나를 움직이는 너의 미...


In [117]:
eigth_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
eigth_df = eigth_df[eigth_df['Year'].isin([2006, 2007, 2008, 2009])]
eigth_df.tail()

,Artist,Genre,Link,Title,Year
3267,손담비,댄스,https://www.melon.com/song/detail.htm?songId=2...,아몰레드 (Amoled),2009
3268,김종국,발라드,https://www.melon.com/song/detail.htm?songId=2...,행복병 (Feat. Shorry J (MM)),2009
3269,마이티 마우스,랩/힙합,https://www.melon.com/song/detail.htm?songId=2...,연애특강 (Feat. 한예슬),2009
3270,유키스,댄스,https://www.melon.com/song/detail.htm?songId=2...,만만하니,2009
3271,럼블피쉬,발라드,https://www.melon.com/song/detail.htm?songId=2...,한사람을 위한 마음,2009


In [118]:
eigth_df['Lyrics'] = eigth_df['Link'].apply(get_lyrics)
eigth_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
3267,손담비,댄스,https://www.melon.com/song/detail.htm?songId=2...,아몰레드 (Amoled),2009,빛나지 그 누구보다더 밝게 더욱 빛나지그 누가 발하는 빛보다 더 진하지비교 조차 할...
3268,김종국,발라드,https://www.melon.com/song/detail.htm?songId=2...,행복병 (Feat. Shorry J (MM)),2009,어떻게 내가 널 찾게 됐는지내가 참 착하게 살아왔는지누구나 한번은 행운이 오나봐넌 ...
3269,마이티 마우스,랩/힙합,https://www.melon.com/song/detail.htm?songId=2...,연애특강 (Feat. 한예슬),2009,수천 번을 말해도 늘 모자라는 말자꾸 자꾸 들어도 듣고 싶은 말마법처럼 우릴 설레이...
3270,유키스,댄스,https://www.melon.com/song/detail.htm?songId=2...,만만하니,2009,U Kiss n BravesoundYou know It's BRAVE내가 그렇게렇게...
3271,럼블피쉬,발라드,https://www.melon.com/song/detail.htm?songId=2...,한사람을 위한 마음,2009,힘들게 보낸 나의 하루에짧은 입맞춤을 해주던 사람언젠간 서로가 더 먼곳을 보며결국엔...


In [130]:
nine_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
nine_df = nine_df[nine_df['Year'].isin([2010, 2011, 2012, 2013])]
nine_df.tail()

,Artist,Genre,Link,Title,Year
3667,버벌진트,랩/힙합,https://www.melon.com/song/detail.htm?songId=4...,비범벅 (Feat. 범키),2013
3668,2NE1,발라드,https://www.melon.com/song/detail.htm?songId=4...,그리워해요,2013
3669,빈지노 (Beenzino),랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,Boogie On & On,2013
3670,버벌진트,랩/힙합,https://www.melon.com/song/detail.htm?songId=4...,시작이 좋아 (Feat. 강민희 Of 미스에스),2013
3671,써니힐,발라드,https://www.melon.com/song/detail.htm?songId=3...,Goodbye To Romance,2013


In [131]:
nine_df['Lyrics'] = nine_df['Link'].apply(get_lyrics)
nine_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
3667,버벌진트,랩/힙합,https://www.melon.com/song/detail.htm?songId=4...,비범벅 (Feat. 범키),2013,yeah VJ yeah Bumkey Let's go 비에 범벅이 돼 이 거리를 거니...
3668,2NE1,발라드,https://www.melon.com/song/detail.htm?songId=4...,그리워해요,2013,그렇게 너무 편하게 굴지 마요아직 너와 난 남남이니까어린애처럼 보채지 좀 마요아직 ...
3669,빈지노 (Beenzino),랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,Boogie On & On,2013,난 괜찮아내일 아침에 소파 위에 시체가되어 이름 모르는 너와 어색한사이가 되어도I ...
3670,버벌진트,랩/힙합,https://www.melon.com/song/detail.htm?songId=4...,시작이 좋아 (Feat. 강민희 Of 미스에스),2013,시작이 좋아조용한 바닷가에 홀로 와 새해 소망도 빌었어한동안 날 괴롭히던 지독한 감...
3671,써니힐,발라드,https://www.melon.com/song/detail.htm?songId=3...,Goodbye To Romance,2013,It's time for goodbye 나의 로맨스여 Goodbye 바로 어제처럼...


In [9]:
ten_df = pd.read_csv('./datas/final2019.csv', index_col=[0])
ten_df = ten_df[ten_df['Year'].isin([2014, 2015, 2016, 2017])]
ten_df.tail()

,Artist,Genre,Link,Title,Year
4067,Apink (에이핑크),댄스,https://www.melon.com/song/detail.htm?songId=3...,FIVE,2017
4068,아이유,발라드,https://www.melon.com/song/detail.htm?songId=3...,이 지금,2017
4069,민서,발라드,https://www.melon.com/song/detail.htm?songId=3...,좋아,2017
4070,수지 (SUZY),R&B/Soul,https://www.melon.com/song/detail.htm?songId=3...,행복한 척,2017
4071,아이오아이 (I.O.I),댄스,https://www.melon.com/song/detail.htm?songId=3...,너무너무너무,2017


In [10]:
ten_df['Lyrics'] = ten_df['Link'].apply(get_lyrics)
ten_df.tail()

,Artist,Genre,Link,Title,Year,Lyrics
4067,Apink (에이핑크),댄스,https://www.melon.com/song/detail.htm?songId=3...,FIVE,2017,와요 와요 와요 와너와 나 지금부터 잠시 12345눈을 감고 가만히 서 12345잠...
4068,아이유,발라드,https://www.melon.com/song/detail.htm?songId=3...,이 지금,2017,이건 비밀이야 아무에게도 고백하지 않았던이야기를 들려주면큰 눈으로 너는 묻지How ...
4069,민서,발라드,https://www.melon.com/song/detail.htm?songId=3...,좋아,2017,이제 괜찮니 너무 힘들었다며 너의 그 마무리가고작 이별뿐일 거라 우린 괜찮다면서잘 ...
4070,수지 (SUZY),R&B/Soul,https://www.melon.com/song/detail.htm?songId=3...,행복한 척,2017,난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...
4071,아이오아이 (I.O.I),댄스,https://www.melon.com/song/detail.htm?songId=3...,너무너무너무,2017,날 너무너무너무 너무너무너무 너무너무너무 너무너무너무 너무너무너무 너무 좋아하면 그...


In [11]:
ten = pd.read_csv('./datas/final2019.csv', index_col=[0])
ten = ten[ten['Year'].isin([2018, 2019])]
ten.tail()

,Artist,Genre,Link,Title,Year
4267,아이유,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,Blueming,2019
4268,MC몽,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,"인기 (Feat. 송가인, 챈슬러)",2019
4269,ITZY (있지),댄스,https://www.melon.com/song/detail.htm?songId=3...,ICY,2019
4270,Lil tachi,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,눈 (Prod. 기리보이),2019
4271,홍진영,성인가요,https://www.melon.com/song/detail.htm?songId=3...,오늘 밤에,2019


In [12]:
ten['Lyrics'] = ten['Link'].apply(get_lyrics)
ten.tail()

,Artist,Genre,Link,Title,Year,Lyrics
4267,아이유,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,Blueming,2019,‘뭐해?‘라는 두 글자에‘네가 보고 싶어’ 나의 속마음을 담아 우이모티콘 하나하나 ...
4268,MC몽,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,"인기 (Feat. 송가인, 챈슬러)",2019,우선 캐릭터는 분명해야 해요 X 같은 XX철부지같이 굴면Fail 해요 Next 바로...
4269,ITZY (있지),댄스,https://www.melon.com/song/detail.htm?songId=3...,ICY,2019,Hey hey hey yass whoo Beep beep Hey hey hey he...
4270,Lil tachi,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,눈 (Prod. 기리보이),2019,멋이 다 흘러흘러서 넘쳐멀어 버렸어배로 막 벌어멋이 다 흘러멋이 다 흘러배로 막 벌...
4271,홍진영,성인가요,https://www.melon.com/song/detail.htm?songId=3...,오늘 밤에,2019,오늘 밤에 아무도 모르게너랑 둘이서 둘이서 사랑을 할거야오늘 밤에 아무도 모르게뚜뚜...


In [132]:
frames = [first_df, second_df, third_df, fourth_df, fifth_df, sixth_df,seventh_df, eigth_df, nine_df]
df = pd.concat(frames)
df

/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Artist,Genre,Link,Lyrics,Title,Year
0,금호동,성인가요,https://www.melon.com/song/detail.htm?songId=4...,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...,내일또 만납시다,1964
1,최숙자,성인가요,https://www.melon.com/song/detail.htm?songId=1...,조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...,눈물의 연평도,1964
2,이미자,성인가요,https://www.melon.com/song/detail.htm?songId=1...,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...,님이라 부르리까,1964
3,이미자,성인가요,https://www.melon.com/song/detail.htm?songId=1...,헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...,동백아가씨,1964
4,현미,성인가요,https://www.melon.com/song/detail.htm?songId=1...,그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...,떠날때는 말없이,1964
5,백야성,성인가요,https://www.melon.com/song/detail.htm?songId=4...,항구의 일~~번지 부기우기 일~번지그라스를 채워~다오 부기우기 아~~가씨고동이 슬피...,마도로스 부기,1964
6,최희준,성인가요,https://www.melon.com/song/detail.htm?songId=1...,눈물도 한숨도나 혼자 씹어 삼키며밤 거리의 뒷골목을누비고 다녀도사랑만은 단 하나의목...,맨발의 청춘,1964
7,권혜경,성인가요,https://www.melon.com/song/detail.htm?songId=5...,-,물새우는 해변,1964
8,박재란,성인가요,https://www.melon.com/song/detail.htm?songId=3...,시원한 밀짚모자포플라 그늘에양떼를 몰고가는목장의 아가씨연분홍 빛 입술에는살며시 웃음...,밀짚모자 목장아가씨,1964
9,현미,성인가요,https://www.melon.com/song/detail.htm?songId=1...,눈을 감고 걸어도눈을 뜨고 걸어도보이는 것은 초라한 모습보고 싶은 얼굴거리마다 물결...,보고싶은 얼굴,1964


In [133]:
df.to_csv('./datas/final2019(rev2).csv')

In [19]:
df = pd.read_csv('./datas/final2019(rev2).csv', index_col=[0])

In [20]:
frames = [df, ten_df, ten]
df = pd.concat(frames)
df.tail()

/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Artist,Genre,Link,Lyrics,Title,Year
4267,아이유,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,‘뭐해?‘라는 두 글자에‘네가 보고 싶어’ 나의 속마음을 담아 우이모티콘 하나하나 ...,Blueming,2019
4268,MC몽,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,우선 캐릭터는 분명해야 해요 X 같은 XX철부지같이 굴면Fail 해요 Next 바로...,"인기 (Feat. 송가인, 챈슬러)",2019
4269,ITZY (있지),댄스,https://www.melon.com/song/detail.htm?songId=3...,Hey hey hey yass whoo Beep beep Hey hey hey he...,ICY,2019
4270,Lil tachi,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,멋이 다 흘러흘러서 넘쳐멀어 버렸어배로 막 벌어멋이 다 흘러멋이 다 흘러배로 막 벌...,눈 (Prod. 기리보이),2019
4271,홍진영,성인가요,https://www.melon.com/song/detail.htm?songId=3...,오늘 밤에 아무도 모르게너랑 둘이서 둘이서 사랑을 할거야오늘 밤에 아무도 모르게뚜뚜...,오늘 밤에,2019


In [21]:
df.to_csv('./datas/final2019(rev2).csv')

In [3]:
df = pd.read_csv('./datas/final2019(rev2).csv', index_col=[0])

In [13]:
df.drop(columns=['Link'], inplace=True)
df

,Artist,Genre,Lyrics,Title,Year
0,금호동,성인가요,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...,내일또 만납시다,1964
1,최숙자,성인가요,조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...,눈물의 연평도,1964
2,이미자,성인가요,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...,님이라 부르리까,1964
3,이미자,성인가요,헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...,동백아가씨,1964
4,현미,성인가요,그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...,떠날때는 말없이,1964
5,백야성,성인가요,항구의 일~~번지 부기우기 일~번지그라스를 채워~다오 부기우기 아~~가씨고동이 슬피...,마도로스 부기,1964
6,최희준,성인가요,눈물도 한숨도나 혼자 씹어 삼키며밤 거리의 뒷골목을누비고 다녀도사랑만은 단 하나의목...,맨발의 청춘,1964
7,권혜경,성인가요,-,물새우는 해변,1964
8,박재란,성인가요,시원한 밀짚모자포플라 그늘에양떼를 몰고가는목장의 아가씨연분홍 빛 입술에는살며시 웃음...,밀짚모자 목장아가씨,1964
9,현미,성인가요,눈을 감고 걸어도눈을 뜨고 걸어도보이는 것은 초라한 모습보고 싶은 얼굴거리마다 물결...,보고싶은 얼굴,1964


In [15]:
df.drop_duplicates(keep='last', inplace=True)

In [16]:
df

,Artist,Genre,Lyrics,Title,Year
0,금호동,성인가요,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...,내일또 만납시다,1964
1,최숙자,성인가요,조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...,눈물의 연평도,1964
2,이미자,성인가요,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...,님이라 부르리까,1964
3,이미자,성인가요,헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...,동백아가씨,1964
4,현미,성인가요,그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...,떠날때는 말없이,1964
5,백야성,성인가요,항구의 일~~번지 부기우기 일~번지그라스를 채워~다오 부기우기 아~~가씨고동이 슬피...,마도로스 부기,1964
6,최희준,성인가요,눈물도 한숨도나 혼자 씹어 삼키며밤 거리의 뒷골목을누비고 다녀도사랑만은 단 하나의목...,맨발의 청춘,1964
7,권혜경,성인가요,-,물새우는 해변,1964
8,박재란,성인가요,시원한 밀짚모자포플라 그늘에양떼를 몰고가는목장의 아가씨연분홍 빛 입술에는살며시 웃음...,밀짚모자 목장아가씨,1964
9,현미,성인가요,눈을 감고 걸어도눈을 뜨고 걸어도보이는 것은 초라한 모습보고 싶은 얼굴거리마다 물결...,보고싶은 얼굴,1964


In [17]:
df.to_csv('./datas/final2019(rev3).csv')